In [7]:
import numpy as np
detailedResults = np.genfromtxt("RegularSeasonDetailedResults.csv", delimiter = ",")
#remove first line
detailedResults = detailedResults[1:]

In [8]:
#Split Data 80-20
cvSize = int(0.2* len(detailedResults))
trainSize = len(detailedResults) - cvSize
trainData = detailedResults[:trainSize]
testData = detailedResults[trainSize:]

In [10]:
schema = {"Season":0, "Daynum":1, "Wteam":2, "Wscore":3, "Lteam":4, "Lscore":5, "Wloc":6, "Numot":7, "Wfgm":8, "Wfga":9, "Wfgm3":10, "Wfga3":11, "Wftm":12, "Wfta":13, "Wor":14, "Wdr":15, "Wast":16, "Wto":17, "Wstl":18,
         "Wblk":19,"Wpf":20,"Lfgm":21,"Lfga":22,"Lfgm3":23, "Lfga3":24, "Lftm":25, "Lfta":26, "Lor":27, "Ldr":28, "Last":29, "Lto":30, "Lstl":31, "Lblk":32, "Lpf":33}

In [63]:
from collections import defaultdict
teamDic = defaultdict( lambda: defaultdict(int))
for data in trainData:
    loctype = data[ schema["Wloc"] ]
    wfgm, lfgm = data[schema["Wfgm"] ], data[schema["Lfgm"]]
    wfga, lfga = data[schema["Wfga"]], data[schema["Lfga"]]
    wfgm3, lfgm3 = data[schema["Wfgm3"]], data[schema["Lfgm3"]]
    wfga3, lfga3 = data[schema["Wfga3"]], data[schema["Lfga3"]]
    wteam, lteam = data[schema["Wteam"]], data[schema["Lteam"]]
    wftm, lftm = data[schema["Wftm"]], data[schema["Lftm"]]
    wfta, lfta = data[schema["Wfta"]], data[schema["Lfta"]]
    if loctype == "N":
        #fg
        teamDic[wteam]["fgmA"] += wfgm
        teamDic[wteam]["fgaA"] += wfga
        teamDic[lteam]["fgmA"] += lfgm
        teamDic[lteam]["fgaA"] += lfga
        #fg3
        teamDic[wteam]["fgm3A"] += wfgm3
        teamDic[wteam]["fga3A"] += wfga3
        teamDic[lteam]["fgm3A"] += lfgm3
        teamDic[lteam]["fga3A"] += lfga3
        #fta
        teamDic[wteam]["ftmA"] += wftm
        teamDic[wteam]["ftaA"] += wfta
        teamDic[lteam]["ftmA"] += lftm
        teamDic[lteam]["ftaA"] += lfta
    elif loctype == "H":
        #fg
        teamDic[wteam]["fgmH"] += wfgm
        teamDic[wteam]["fgaH"] += wfga
        teamDic[lteam]["fgmA"] += lfgm
        teamDic[lteam]["fgaA"] += lfga
        #fg3
        teamDic[wteam]["fgm3H"] += wfgm3
        teamDic[wteam]["fga3H"] += wfga3
        teamDic[lteam]["fgm3A"] += lfgm3
        teamDic[lteam]["fga3A"] += lfga3
        #ft
        teamDic[wteam]["ftmH"] += wftm
        teamDic[wteam]["ftaH"] += wfta
        teamDic[lteam]["ftmA"] += lftm
        teamDic[lteam]["ftaA"] += lfta

    else:
        #fg
        teamDic[wteam]["fgmA"] += wfgm
        teamDic[wteam]["fgaA"] += wfga
        teamDic[lteam]["fgmH"] += lfgm
        teamDic[lteam]["fgaH"] += lfga
        #fg3
        teamDic[wteam]["fgm3H"] += wfgm3
        teamDic[wteam]["fga3H"] += wfga3
        teamDic[lteam]["fgm3A"] += lfgm3
        teamDic[lteam]["fga3A"] += lfga3
        #ft
        teamDic[wteam]["ftmA"] += wftm
        teamDic[wteam]["ftaA"] += wfta
        teamDic[lteam]["ftmH"] += lftm
        teamDic[lteam]["ftaH"] += lfta
    teamDic[wteam]["totalMatches"] += 1
    teamDic[lteam]["totalMatches"] += 1
    teamDic[wteam]["wonCount"] += 1

In [64]:
for team in teamDic:
    try:
        #Add efficiency features
        teamDic[ team ]["fgeA"] = float(teamDic[team]["fgmA"])/(teamDic[team]["fgaA"] or 1)
        teamDic[ team ]["fgeH"] = float(teamDic[team]["fgmH"])/(teamDic[team]["fgaH"]or 1)
        teamDic[ team ]["fgTotE"] = float(teamDic[team]["fgmA"]+teamDic[team]["fgmH"])/(teamDic[team]["fgaA"]+teamDic[team]["fgaH"])
        
        teamDic[ team ]["fge3A"] = float(teamDic[team]["fgm3A"])/(teamDic[team]["fga3A"] or 1)
        teamDic[ team ]["fge3H"] = float(teamDic[team]["fgm3H"])/(teamDic[team]["fga3H"] or 1 )
        teamDic[ team ]["fg3TotE"] = float(teamDic[team]["fgm3A"] + teamDic[team]["fgm3H"])/(teamDic[team]["fga3A"] + teamDic[team]["fga3H"])
        
        teamDic[ team ]["fteA"] = float(teamDic[team]["ftmA"])/(teamDic[team]["ftaA"] or 1)
        teamDic[ team ]["fteH"] = float(teamDic[team]["ftmH"])/(teamDic[team]["ftaH"] or 1)
        teamDic[ team ]["ftTotE"] = float(teamDic[team]["ftmA"]+teamDic[team]["ftmH"])/(teamDic[team]["ftaA"] + teamDic[team]["ftaH"])
        
        #Add average goals made per match:
        teamDic[team]["fgAvg"] = float(teamDic[team]["fgmA"]+teamDic[team]["fgmH"]) / teamDic[team]["totalMatches"]
        teamDic[team]["fg3Avg"] = float(teamDic[team]["fgm3A"]+teamDic[team]["fgm3H"]) / teamDic[team]["totalMatches"]
        teamDic[team]["ftAvg"] = float(teamDic[team]["ftmA"]+teamDic[team]["ftmH"]) / teamDic[team]["totalMatches"]
        
        #Add winnig Probability:
        teamDic[team]["winProb"] = float(teamDic[team]["wonCount"])/teamDic[team]["totalMatches"]
    except:
        print teamDic[team]

In [65]:
import random
trainMatrix, trainLabels = [], []
testMatrix, testLabels = [], []

for data in trainData:
    wteam, lteam = data[schema["Wteam"]], data[schema["Lteam"]]
    rn = random.randint(0,1)
    if rn == 1:
        temp = teamDic[wteam].values() + teamDic[lteam].values()
        trainLabels.append(0)
    else:
        temp = teamDic[lteam].values() + teamDic[wteam].values()
        trainLabels.append(1)    
    trainMatrix.append(temp)

for data in testData:
    wteam, lteam = data[schema["Wteam"]], data[schema["Lteam"]]
    rn = random.randint(0,1)
    if rn == 1:
        temp = teamDic[wteam].values() + teamDic[lteam].values()
        testLabels.append(0)
    else:
        temp = teamDic[lteam].values() + teamDic[wteam].values()
        testLabels.append(1)    
    testMatrix.append(temp)


Build Logistic Regression Model:

In [66]:
from sklearn.linear_model import LogisticRegression
logisticModel = LogisticRegression()
logisticModel.fit(trainMatrix, trainLabels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

Build Decistion Tree Model:

In [67]:
from sklearn.tree import DecisionTreeClassifier
dtModel = DecisionTreeClassifier()
dtModel.fit(trainMatrix, trainLabels)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [68]:
def accuracy(model, testMatrix, testLabels):
    opLabels = model.predict(testMatrix)
    correct = sum(1 for _ in range(0,len(testLabels)) if testLabels[_] == opLabels[_])
    return float(correct)/len(opLabels) * 100 

In [71]:
accuracy( logisticModel, testMatrix, testLabels)

61.27175743964065

In [72]:
accuracy( dtModel, testMatrix, testLabels)

58.96266142616508